Refer from <br>
https://github.com/BY571/Munchausen-RL/blob/master/M-DQN.ipynb  <br>
https://gitlab.ilabt.imec.be/ahadifar/google-research/-/blob/master/munchausen_rl/agents/m_dqn.py


In [1]:
# Find RL_Note path and append sys path
import os, sys
cwd = os.getcwd()
pos = cwd.find('RL_Note')
root_path = cwd[0:pos] + 'RL_Note'
sys.path.append(root_path)

In [2]:
import gym
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from pys.utils.memory import ReplayMemory

In [3]:
class DQN(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = Dense(255,activation='relu')
        self.fc2 = Dense(255,activation='relu')
        self.out = Dense(action_size,kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))

    def call(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.out(x)
        return q

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper-parameters for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.entropy_tau = 0.03
        self.alpha = 0.9        
        self.lo = -1
        # Experience Replay
        self.batch_size = 8
        self.train_start = 500
        self.buffer_size = 10000
        self.memory = ReplayMemory(capacity=self.buffer_size)
        
        # Neural Network Architecture
        self.model        = DQN(self.state_size, self.action_size)
        self.target_model = DQN(self.state_size, self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.update_target_model()
        self.show_media_info = False
        
    def remember(self, state, action, reward, next_state, done):
        state = np.reshape(state, [1, self.state_size])
        next_state = np.reshape(next_state, [1, self.state_size])
        transition = (state, action, reward, next_state, done)
        self.memory.append(transition)
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def choose_action(self,state):
        state = np.reshape(state, [1, self.state_size])
        # Exploration and Exploitation
        if (np.random.rand() <= self.epsilon):
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state))

    def get_action(self, state):
        return np.argmax(self.model.predict(state))
        
    def get_target_action(self, state):
        return np.argmax(self.target_model.predict(state))
        
    def train_model(self):
        # Train from Experience Replay
        # Training Condition - Memory Size
        if len(self.memory) < self.train_start:
            return 0.0
        # Decaying Exploration Ratio
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        # Sampling from the memory
        mini_batch = self.memory.sample(self.batch_size)
        
        states      = tf.convert_to_tensor(np.array([sample[0] for sample in mini_batch]))
        actions     = tf.convert_to_tensor(np.array([sample[1] for sample in mini_batch]))
        rewards     = tf.convert_to_tensor(np.array([sample[2] for sample in mini_batch]),dtype=tf.float32)
        rewards     = tf.expand_dims(rewards, axis = 1)
        next_states = tf.convert_to_tensor(np.array([sample[3] for sample in mini_batch]))
        dones       = tf.convert_to_tensor(np.array([sample[4] for sample in mini_batch]),dtype=tf.float32)
        dones       = tf.expand_dims(dones, axis = 1)
        
        if self.show_media_info == False:
            self.show_media_info = True
            print('Start to train, check batch shapes')
            print('shape of states', np.shape(states),type(states))
            print('shape of actions', np.shape(actions),type(actions))
            print('shape of rewards', np.shape(rewards),type(rewards))
            print('shape of next_states', np.shape(next_states),type(next_states))
            print('shape of dones', np.shape(dones),type(dones))
        
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            # munchausen reward
            target_q = tf.stop_gradient(self.target_model(states))
            target_v = tf.reduce_max(target_q,axis=1)
            target_v = tf.expand_dims(target_v,axis=1)

            logsum = tf.math.reduce_logsumexp(((target_q - target_v)/self.entropy_tau),axis=1)
            logsum = tf.expand_dims(logsum,axis=1)
            log_pi = target_q - target_v - self.entropy_tau * logsum
            one_hot_action = tf.one_hot(actions, self.action_size)
            log_pi = tf.reduce_sum(one_hot_action * log_pi, axis=1)
            log_pi = tf.expand_dims(log_pi,axis=1)
            
            # tau_log_pi = target_q - np.amax(target_q, axis=-1)
            term_1st = rewards + self.alpha * self.entropy_tau * tf.clip_by_value(log_pi, clip_value_min=self.lo, clip_value_max=0.0)

            # munchausen target value y
            target_next_q = tf.stop_gradient(self.target_model(next_states))
            target_next_v = tf.reduce_max(target_next_q,axis=1)
            target_next_v = tf.expand_dims(target_next_v,axis=1)
            next_logsum = tf.math.reduce_logsumexp(((target_next_q - target_next_v)/self.entropy_tau),axis=1)
            next_logsum = tf.expand_dims(next_logsum,axis=1)
            target_next_log_pi = target_next_q - target_next_v - self.entropy_tau * next_logsum
            target_pi = tf.math.softmax(target_next_q/self.entropy_tau,1)
            

            term_2nd = self.discount_factor * \
                tf.reduce_sum(target_pi * (target_next_q - target_next_log_pi) * (1 - dones), axis=1)

            target_y = term_1st + term_2nd

            # get q value
            q = self.model(states)
            one_hot_action = tf.one_hot(actions, self.action_size)
            q = tf.reduce_sum(one_hot_action * q, axis=1)
            
            # Compute loss
            loss = tf.reduce_mean(tf.square(target_y - q))
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        return loss

    def save_model(self):
        self.model.save_weights('./save_model/cartpole_dqn_M_TF', save_format='tf')

In [5]:
%matplotlib tk

ENV_NAME = 'CartPole-v1'
EPISODES = 1000
END_SCORE = 400

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = DQNAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores_avg, scores_raw, episodes, losses = [], [], [], []
    epsilons = []
    score_avg = 0
    
    end = False
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []

        state = env.reset()
        
        while not done:
            #env.render()

            # Interact with env.
            action = agent.choose_action(state)
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            loss = agent.train_model()
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                agent.update_target_model()

                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | mem length: {:4d} | epsilon: {:.4f}'
                      .format(e, score_avg, len(agent.memory), agent.epsilon))

                # Save data for plot
                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                losses.append(np.mean(loss_list))
                epsilons.append(agent.epsilon)

                # View data
                plt.clf()
                plt.subplot(311)
                plt.plot(episodes, scores_avg, 'b')
                plt.plot(episodes, scores_raw, 'b', alpha=0.8, linewidth=0.5)
                plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                plt.title('LunarLanderv2 DQN-Munchausen')
                plt.subplot(312)
                plt.plot(episodes, epsilons, 'b')
                plt.xlabel('episode'); plt.ylabel('epsilon'); plt.grid()
                plt.subplot(313)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode'); plt.ylabel('losses') ;plt.grid()
                
                plt.savefig('./result/N255_B8/cartpole_dqn_M_TF.png')

                if score_avg > END_SCORE:
                    agent.save_model()
                    end = True
                    break
        if end == True:
            env.close()
            np.save('./result/N255_B8/cartpole_dqn_M_TF_epi',  episodes)
            np.save('./result/N255_B8/cartpole_dqn_M_TF_scores_avg',scores_avg)
            np.save('./result/N255_B8/cartpole_dqn_M_TF_scores_raw',scores_raw)
            np.save('./result/N255_B8/cartpole_dqn_M_TF_loss', losses)
            print("End")
            break

Env Name :  CartPole-v1
States 4, Actions 2
epi:   0 | score avg 18.00 | mem length:   18 | epsilon: 1.0000
epi:   1 | score avg 18.90 | mem length:   45 | epsilon: 1.0000
epi:   2 | score avg 20.01 | mem length:   75 | epsilon: 1.0000
epi:   3 | score avg 19.91 | mem length:   94 | epsilon: 1.0000
epi:   4 | score avg 19.82 | mem length:  113 | epsilon: 1.0000
epi:   5 | score avg 19.14 | mem length:  126 | epsilon: 1.0000
epi:   6 | score avg 19.12 | mem length:  145 | epsilon: 1.0000
epi:   7 | score avg 18.71 | mem length:  160 | epsilon: 1.0000
epi:   8 | score avg 18.24 | mem length:  174 | epsilon: 1.0000
epi:   9 | score avg 18.42 | mem length:  194 | epsilon: 1.0000
epi:  10 | score avg 19.47 | mem length:  223 | epsilon: 1.0000
epi:  11 | score avg 20.43 | mem length:  252 | epsilon: 1.0000
epi:  12 | score avg 19.98 | mem length:  268 | epsilon: 1.0000
epi:  13 | score avg 21.59 | mem length:  304 | epsilon: 1.0000
epi:  14 | score avg 21.63 | mem length:  326 | epsilon: 1.0